In [155]:
import numpy as np
import pandas as pd
import string
import re
import csv
import pprint as pp

df_remotes = pd.read_csv('turnstile_location.csv', delimiter = ',', header = 0, skipinitialspace = True)

df_remotes.drop('1', axis = 1, inplace = True)

print(df_remotes)

    Remote  Booth          Station    Line Name Division
0     R001   A060     WHITEHALL ST           R1      BMT
1     R001   A058     WHITEHALL ST           R1      BMT
2     R001  R101S      SOUTH FERRY           R1      IRT
3     R002   A077        FULTON ST     ACJZ2345      BMT
4     R002   A081        FULTON ST     ACJZ2345      BMT
5     R002   A082        FULTON ST     ACJZ2345      BMT
6     R003   J025    CYPRESS HILLS            J      BMT
7     R004   J028     ELDERTS LANE           JZ      BMT
8     R005   J030   FOREST PARKWAY            J      BMT
9     R006   J031   WOODHAVEN BLVD           JZ      BMT
10    R006   J032   WOODHAVEN BLVD           JZ      BMT
11    R007   J034           104 ST           JZ      BMT
12    R008   J035           111 ST            J      BMT
13    R009   J037           121 ST           JZ      BMT
14    R010  N062A  42 ST-PA BUS TE  ACENQRS1237      IND
15    R010   N060  42 ST-PA BUS TE  ACENQRS1237      IND
16    R011  N063A  42 ST-PA BUS

In [156]:
df_stations = pd.read_csv('stations.csv', delimiter = ',', header = 0, skipinitialspace = True)

print(df_stations)

     Station ID  Complex ID GTFS Stop ID Division                 Line  \
0             1           1          R01      BMT              Astoria   
1             2           2          R03      BMT              Astoria   
2             3           3          R04      BMT              Astoria   
3             4           4          R05      BMT              Astoria   
4             5           5          R06      BMT              Astoria   
5             6           6          R08      BMT              Astoria   
6             7         613          R11      BMT              Astoria   
7             8           8          R13      BMT              Astoria   
8             9           9          R14      BMT  Broadway - Brighton   
9            10          10          R15      BMT  Broadway - Brighton   
10           11         611          R16      BMT  Broadway - Brighton   
11           12         607          R17      BMT  Broadway - Brighton   
12           13          13          R

In [195]:
from datetime import timedelta, date
import datetime
import requests
import string

start_date = datetime.date(2018, 3, 31)
end_date = datetime.date(2018, 4, 7)

date_year = str(start_date.year)
date_month = str(start_date.month)
date_day = str(start_date.day)

data_url_date = date_year[2:] + date_month.zfill(2) + date_day.zfill(2)
data_url_base = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_'
data_url_ext = '.txt'
data_url = data_url_base + data_url_date + data_url_ext

# initialize an empty dataframe to store data inside
df_traffic = pd.DataFrame()

# define column names
cols_traffic = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']

while start_date <= end_date:
    df_loop = pd.DataFrame()
    
    date_year = str(start_date.year)
    date_month = str(start_date.month)
    date_day = str(start_date.day)

    data_url_date = date_year[2:] + date_month.zfill(2) + date_day.zfill(2)

    data_url = data_url_base + data_url_date + data_url_ext
    # print(data_url)
    
    if requests.get(data_url).status_code == 200:
        df_loop = pd.read_table(data_url, delimiter = ',', names = cols_traffic, skiprows = 1, skipinitialspace = True, lineterminator='\n')
        df_traffic = pd.concat([df_traffic, df_loop])
    else: print('Failed')
        
    # increment date by 7 days to run in next loop iteration
    start_date = start_date + datetime.timedelta(days = 7)
    
print(df_traffic.head(10))

    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/24/2018  00:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/24/2018  04:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/24/2018  08:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/24/2018  12:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/24/2018  16:00:00   
5  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/24/2018  20:00:00   
6  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/25/2018  00:00:00   
7  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/25/2018  04:00:00   
8  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/25/2018  08:00:00   
9  A002  R051  02-00-00   59 ST  NQR456W      BMT  03/25/2018  12:00:00   

      DESC  ENTRIES    EXITS  
0  REGULAR  6559322  2221550  
1  REGULAR  6559322  2221550  
2  REGULAR  6559322  2221550  
3  REGULAR  6559323  2221551  
4  REGULAR  6559327

In [184]:
data_url_base = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_'
data_url_date = ''
data_url = data_url_base + data_url_date
data_url_ext = '.txt'

cols_traffic = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']

# initialize an empty dataframe to store data inside
df_traffic = pd.DataFrame(columns = cols_traffic)

df_loop = pd.DataFrame(columns = cols_traffic)

start_date = datetime.date(2015, 1, 3)
end_date = datetime.date(2015, 1, 10)
date_year = str(start_date.year)
date_month = str(start_date.month)
date_day = str(start_date.day)
data_url_date = date_year[2:] + date_month.zfill(2) + date_day.zfill(2)
data_url = data_url_base + data_url_date + data_url_ext

df_loop = pd.read_table(data_url, delimiter = ',', names = cols_traffic, skiprows = 1, skipinitialspace = True, lineterminator='\n')
df.columns = df.columns.str.replace('\s+', '')
df_traffic = pd.concat([df_traffic, df_loop])

In [197]:
df_traffic

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2018,00:00:00,REGULAR,6559322,2221550
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2018,04:00:00,REGULAR,6559322,2221550
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2018,08:00:00,REGULAR,6559322,2221550
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2018,12:00:00,REGULAR,6559323,2221551
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2018,16:00:00,REGULAR,6559327,2221555
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2018,20:00:00,REGULAR,6559341,2221558
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/25/2018,00:00:00,REGULAR,6559565,2221604
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/25/2018,04:00:00,REGULAR,6559565,2221604
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/25/2018,08:00:00,REGULAR,6559565,2221604
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/25/2018,12:00:00,REGULAR,6559568,2221606


In [185]:
df_traffic.isnull().sum()

C/A         0
UNIT        0
SCP         0
STATION     0
LINENAME    0
DIVISION    0
DATE        0
TIME        0
DESC        0
ENTRIES     0
EXITS       0
dtype: int64

In [186]:
print(list(df_traffic.columns.values))

# df_traffic.EXITS.head(1)

# df_traffic['TOT_MOVEMENT'] = df.apply(lambda row: row['ENTRIES'] + row['EXITS'], axis=1)

['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS']


In [191]:
find_reset = (df_traffic.ENTRIES == 0) & (df_traffic.EXITS == 0)

In [188]:
df_traffic

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/27/2014,07:00:00,REGULAR,4936852,1672762
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/27/2014,11:00:00,REGULAR,4936909,1672839
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/27/2014,15:00:00,REGULAR,4937131,1672892
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/27/2014,19:00:00,REGULAR,4937588,1672949
5,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/27/2014,23:00:00,REGULAR,4937823,1672997
6,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/28/2014,03:00:00,REGULAR,4937911,1673010
7,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/28/2014,07:00:00,REGULAR,4937920,1673018
8,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/28/2014,11:00:00,REGULAR,4937960,1673068
9,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/28/2014,15:00:00,REGULAR,4938101,1673127
10,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,12/28/2014,19:00:00,REGULAR,4938365,1673196


In [ ]:
df_traffic.groupby('C/A','UNIT','STATION','LINENAME','DIVISION','DATE').get_group()